## Twitter Sentiment Analisys menggunakan metode A NAIVE BAYES CLASSIFIER (NBC)
### 1.Case Folding:
Case folding dilakukan untuk
mengubah setiap karakter didalam teks menjadi huruf
kecil. Tidak semua kata dalam teks konsisten dalam
penggunaan huruf kapital disinilah tujuan dilakukan case
folding untuk mengkonversi setiap karakter dalam kata
menjadi huruf kecil.
### 2.Tokenisasi: 
Tokenisasi merupakan proses
pemecahan kata pada suatu teks ke dalam satuan kata.
Tokenisasi dilakukan untuk menghasilkan kumpulan kata
yang berdiri sendiri, tokenisasi memecah teks yang
semula berupa kalimat menjadi kata-kata. tokenisasi
menghilangkan delimeter seperti titik (.), koma (,), spasi,
dan karakter angka yang ada pada kata tersebut.
Dalam penelitian ini tokenisasi dilakukan untuk
memecah kata, serta melakukan penghapusan delimeter
beserta karakter angka bersama tweet entity seperti
hashtag, retweet dan mention.
### 3.Filtering: 
Filtering merupakan proses dalam text
preprocessing setelah tokenisasi, filtering dilakukan untuk
untuk mengambil kata penting hasil tokenisasi. Pada tahap
filtering kata akan ditentukan apakah akan digunakan atau 
dibuang. Proses dalam filtering dalam membuang katakata yang tidak digunakan atau stopword terdapat dalam
bag of words stoplist.
Stopword merupakan daftar kata-kata yang tidak
mempresentasikan isi dari suatu dokumen teks, stopword
dilakukan untuk meghilangkan kata atau term yang tidak
memiliki arti. Daftar stoplist akan dibuat sebelum
melakukan proses stopword removal, jika kata-kata
terdapat dalam daftar stoplist, maka kata tersebut akan
dihapus, sehingga kata-kata yang tersisa akan dianggap
kata yang mencirikan isi suatu dokumen.
### 4.Stemming: 
Stemming merupakan proses mengubah
kata menjadi bentuk dasarnya. Stemming dilakukan untuk
meyeragamkan bentuk kata. Tujuan dari proses stemming
adalah menghilangkan imbuhan-imbuhan baik itu berupa
prefiks, sufiks, maupun konfiks yang ada pada setiap kata. 
Stemming dalam penelitian ini dilakukan berdasarkan
aturan morfologi bahasa Indonesia.


### berikut adalah langka-langkahnya:

### 1.Install Library(module) yang dibutuhkan

In [31]:
# %pip install tweepy
# %pip install Sastrawi
# %pip install numpy
# %pip install sklearn
# %pip install textblob
# %pip install pandas
# %pip install matplotlib

In [32]:
import tweepy

BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAJQrYQEAAAAA1BtlexYIrHRZvqkz6TEOraEC%2B0k%3D5cOp6U7ytEMpcTDctDQWVl6HFd0Y28pxhDnTGyHA9KiiC1MVCF'

client = tweepy.Client(bearer_token=BEARERTOKEN)

query = '"Elon Musk" lang:id'

# get data max 100
responses = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at'])
tweet100 = [tweet for tweet in responses.data]

# get max 1000
tweettexts = [tweet.text for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=1000)]

In [33]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
import re

In [34]:
data = {
    'text' : tweettexts
}
df = pd.DataFrame(data)
df.to_csv(f"crawling.csv", mode='w', index=False, header=data.keys())

In [35]:
import pandas as pd

df = pd.read_csv('crawling.csv', encoding='unicode_escape')
df.head(5)

,text
0,ga jelas bngt ni elon musk https://t.co/nhNGZv...
1,Elon Musk Bakal Batalkan Proses Akuisisi Twitt...
2,Elon Musk kabarnya ingin memecat 10 persen kar...
3,Elon Musk Ancam Bakal Batalkan Pembelian Twitt...
4,W lazm bill gates w elon musk w mark zuckerber...


In [36]:
#Cleaning Text
def textcleaner(text):
    new_text = str(text)
    
    # remove old style retweet text "RT"
    new_text = re.sub(r'^RT\s+', '', new_text)
    
    # remove username
    new_text = re.sub(r'@([A-Za-z0-9_]+)', '', new_text)

    # remove hyperlinks
    new_text = re.sub(r'https?:\/\/.*[\r\n]*', '', new_text)

    # remove invalid character
    new_text = re.sub("[^A-Za-z" "]+", " ", new_text)
    
    new_text    = re.sub(r'\b\w(1,2)\b',' ', new_text) #menghilangkan 2 kata
    
    return new_text

In [37]:
def caselower(sentences):
    return sentences.lower()

In [38]:
def tokenmaker(sentences):
    return sentences.strip().split()

In [39]:
#Stop Removal
stopword = nltk.corpus.stopwords.words("indonesian")
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [40]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

In [41]:
# create labeler

pos_kata = [pos.strip() for pos in open('kata_positif_2.txt', 'r').readlines()]
neg_kata = [neg.strip() for neg in open('kata_negatif_2.txt', 'r').readlines()]

#list kata-kata negasi
negasi = [neg.strip() for neg in open('negasi.txt', 'r').readlines()]


def labeling(stemmed):
    count_pos = 0
    count_neg = 0
    for ind, stem in enumerate(stemmed):
        if stem.strip() in pos_kata:
            if stemmed[ind-1].strip() in negasi:
                count_neg += 1
            else:
                count_pos += 1
        elif stem.strip() in neg_kata:
            if stemmed[ind-1].strip() in negasi:
                count_pos += 1
            else:
                count_neg += 1

    return 'NETRAL' if count_pos == count_neg else 'POSITIF' if count_pos > count_neg else 'NEGATIF'
    


In [42]:
# cleaning text
df['clean_text'] = df['text'].apply(lambda x : textcleaner(x))

In [43]:
# convert ke lower case
df['lower'] = df['clean_text'].apply(lambda x : caselower(x))

In [44]:
# tokenizing
df['token'] = df['lower'].apply(lambda x : tokenmaker(x))

In [45]:
# stopword removing
df["stop_removed"] = df['token'].apply(lambda x: remove_stopwords(x))

In [46]:
# stemming
df["stemmed"] = df['stop_removed'].apply(lambda x: stemming(x))
print("Stemming selesai!")

Stemming selesai!


In [51]:
# labelling sentiment
df["label"] = df['stemmed'].apply(lambda x: labeling(x))
df.to_csv(f"labeled.csv", mode='w', index=False, header=True)
df.head(df.size)

,text,clean_text,lower,token,stop_removed,stemmed,label,polarity
0,ga jelas bngt ni elon musk https://t.co/nhNGZv...,ga jelas bngt ni elon musk,ga jelas bngt ni elon musk,"[ga, jelas, bngt, ni, elon, musk]","[ga, bngt, ni, elon, musk]","[ga, bngt, ni, elon, musk]",NETRAL,0
1,Elon Musk Bakal Batalkan Proses Akuisisi Twitt...,Elon Musk Bakal Batalkan Proses Akuisisi Twitter,elon musk bakal batalkan proses akuisisi twitter,"[elon, musk, bakal, batalkan, proses, akuisisi...","[elon, musk, batalkan, proses, akuisisi, twitter]","[elon, musk, batal, proses, akuisisi, twitter]",NETRAL,0
2,Elon Musk kabarnya ingin memecat 10 persen kar...,Elon Musk kabarnya ingin memecat persen karyaw...,elon musk kabarnya ingin memecat persen karyaw...,"[elon, musk, kabarnya, ingin, memecat, persen,...","[elon, musk, kabarnya, memecat, persen, karyaw...","[elon, musk, kabar, pecat, persen, karyawannya...",NEGATIF,-1
3,Elon Musk Ancam Bakal Batalkan Pembelian Twitt...,Elon Musk Ancam Bakal Batalkan Pembelian Twitter,elon musk ancam bakal batalkan pembelian twitter,"[elon, musk, ancam, bakal, batalkan, pembelian...","[elon, musk, ancam, batalkan, pembelian, twitter]","[elon, musk, ancam, batal, beli, twitter]",NEGATIF,-1
4,W lazm bill gates w elon musk w mark zuckerber...,W lazm bill gates w elon musk w mark zuckerber...,w lazm bill gates w elon musk w mark zuckerber...,"[w, lazm, bill, gates, w, elon, musk, w, mark,...","[w, lazm, bill, gates, w, elon, musk, w, mark,...","[w, lazm, bill, gates, w, elon, musk, w, mark,...",NETRAL,0
...,...,...,...,...,...,...,...,...
995,RT @ArieMutyara: Pantesan Elon Musk ogahÂ²an i...,Pantesan Elon Musk ogah an inpes krn dia tau ...,pantesan elon musk ogah an inpes krn dia tau ...,"[pantesan, elon, musk, ogah, an, inpes, krn, d...","[pantesan, elon, musk, ogah, an, inpes, krn, t...","[pantesan, elon, musk, ogah, an, inpes, krn, t...",NETRAL,0
996,"RT @detikcom: Mahasiswa asal Jember, Kevin, be...",Mahasiswa asal Jember Kevin berhasil diterima...,mahasiswa asal jember kevin berhasil diterima...,"[mahasiswa, asal, jember, kevin, berhasil, dit...","[mahasiswa, jember, kevin, berhasil, diterima,...","[mahasiswa, jember, kevin, hasil, terima, usah...",POSITIF,1
997,"@mujiza_t @FeniRose_ Cebong nyinyirin FE, pdah...",Cebong nyinyirin FE pdahal junjunganya sibuk ...,cebong nyinyirin fe pdahal junjunganya sibuk ...,"[cebong, nyinyirin, fe, pdahal, junjunganya, s...","[cebong, nyinyirin, fe, pdahal, junjunganya, s...","[cebong, nyinyirin, fe, pdahal, junjunganya, s...",POSITIF,1
998,"RT @detikcom: Mahasiswa asal Jember, Kevin, be...",Mahasiswa asal Jember Kevin berhasil diterima...,mahasiswa asal jember kevin berhasil diterima...,"[mahasiswa, asal, jember, kevin, berhasil, dit...","[mahasiswa, jember, kevin, berhasil, diterima,...","[mahasiswa, jember, kevin, hasil, terima, usah...",POSITIF,1


In [52]:
import pandas as pd
import re
import numpy as np
import joblib
import pickle
import csv
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [49]:
# convert label to polarity
def convert(polarity):
    if polarity == 'POSITIF':
        return 1
    elif polarity == 'NETRAL':
        return 0
    else:
        return -1

In [53]:
df = pd.read_csv('labeled.csv')
df['polarity'] = df['label'].apply(lambda x: convert(x))
df.head(5)

,text,clean_text,lower,token,stop_removed,stemmed,label,polarity
0,ga jelas bngt ni elon musk https://t.co/nhNGZv...,ga jelas bngt ni elon musk,ga jelas bngt ni elon musk,"['ga', 'jelas', 'bngt', 'ni', 'elon', 'musk']","['ga', 'bngt', 'ni', 'elon', 'musk']","['ga', 'bngt', 'ni', 'elon', 'musk']",NETRAL,0
1,Elon Musk Bakal Batalkan Proses Akuisisi Twitt...,Elon Musk Bakal Batalkan Proses Akuisisi Twitter,elon musk bakal batalkan proses akuisisi twitter,"['elon', 'musk', 'bakal', 'batalkan', 'proses'...","['elon', 'musk', 'batalkan', 'proses', 'akuisi...","['elon', 'musk', 'batal', 'proses', 'akuisisi'...",NETRAL,0
2,Elon Musk kabarnya ingin memecat 10 persen kar...,Elon Musk kabarnya ingin memecat persen karyaw...,elon musk kabarnya ingin memecat persen karyaw...,"['elon', 'musk', 'kabarnya', 'ingin', 'memecat...","['elon', 'musk', 'kabarnya', 'memecat', 'perse...","['elon', 'musk', 'kabar', 'pecat', 'persen', '...",NEGATIF,-1
3,Elon Musk Ancam Bakal Batalkan Pembelian Twitt...,Elon Musk Ancam Bakal Batalkan Pembelian Twitter,elon musk ancam bakal batalkan pembelian twitter,"['elon', 'musk', 'ancam', 'bakal', 'batalkan',...","['elon', 'musk', 'ancam', 'batalkan', 'pembeli...","['elon', 'musk', 'ancam', 'batal', 'beli', 'tw...",NEGATIF,-1
4,W lazm bill gates w elon musk w mark zuckerber...,W lazm bill gates w elon musk w mark zuckerber...,w lazm bill gates w elon musk w mark zuckerber...,"['w', 'lazm', 'bill', 'gates', 'w', 'elon', 'm...","['w', 'lazm', 'bill', 'gates', 'w', 'elon', 'm...","['w', 'lazm', 'bill', 'gates', 'w', 'elon', 'm...",NETRAL,0


In [ ]:
x = df['stop_removed']
y = df['polarity']

In [68]:
bow_transformer = CountVectorizer()
x = bow_transformer.fit_transform(df['stop_removed'])
print(x.toarray())
print(x.shape)
print(x.nnz)

# save count vectorized to disk
filename = 'count_vectorize.pkl'
pickle.dump(bow_transformer, open(filename, 'wb'))

# TFID Transform
tf_transform = TfidfTransformer(use_idf=False).fit(x)
x = tf_transform.transform(x)
print(x.shape)

# save TFID to disk
filename = 'tfid_transform.pkl'
pickle.dump(tf_transform, open(filename, 'wb'))


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1000, 2960)
11864
(1000, 2960)
